In [28]:
from tqdm import tqdm
import numpy as np

In [57]:
# name of file exported from ensembl
FILE_NAME = "mart_export.txt"
file = open(FILE_NAME, "r")

lines_to_read = 1000000
all_introns = []
curr_intron = []

last_intron = ''

# count number of lines for tqdm
num_lines = sum(1 for line in open(FILE_NAME,'r'))

for i in tqdm(range(num_lines)):
    
    line = file.readline()
    if line[0] == '>': # new gene
        
        # put last intron sequence into previous gene
        if len(last_intron) != 0:#len(last_intron) == tr_size:
            curr_intron[-1] = last_intron
            all_introns.append(curr_intron)
            
        last_intron = ''
        
        l = line[1:-1].split('|') # remove '>'
        
        # get gene and transcript information
        gene_id, gene_id_version, transcript_id = l[0], l[1], l[3]
        tr_start, tr_end = int(l[6]), int(l[7])
        strand, tr_size = int(l[10]), int(l[11])

        # take and order exons
        exon_starts = [int(bp) for bp in l[8].split(';')]
        exon_ends = [int(bp) for bp in l[9].split(';')]
        order = np.argsort(exon_starts)
        exon_starts = np.take(exon_starts, order)
        exon_ends = np.take(exon_ends, order)
        
        curr_intron = [gene_id, gene_id_version, transcript_id, tr_start,
                      tr_end, strand, tr_size, exon_starts, exon_ends, '']
        
        
    else:
        last_intron += line[:-1]

# do last one manually
curr_intron[-1] = last_intron
all_introns.append(curr_intron)
        
print('Done processing. Found', len(all_introns), 'gene transcripts.')
        

100%|██████████| 153670579/153670579 [03:15<00:00, 785090.51it/s]

Done processing. Found 249606 gene transcripts.


In [63]:
all_genes = list(set([t[0] for t in all_introns]))

In [70]:
# We want to find the best transcript for each gene, since each one
# has multiple transcripts. we choose the best one by its exon size
best_transcripts = {}
for i in tqdm(range(len(all_introns)), position=0, leave=True):
    intron = all_introns[i]
    gene = intron[0]
    tr_size = intron[6]
    if gene in best_transcripts:
        if tr_size > best_transcripts[gene][0]:
            best_transcripts[gene] = (tr_size, intron)
    else:
        best_transcripts[gene] = (tr_size, intron)


100%|██████████| 249606/249606 [00:02<00:00, 84543.23it/s]


In [71]:
# number of unique genes
len(best_transcripts)

67140

In [227]:
# Sanity check
best_transcripts['ENSG00000147642'][1][:-1]

['ENSG00000147642',
 'ENSG00000147642.17',
 'ENST00000399066.7',
 109574182,
 109643675,
 -1,
 3435,
 array([109574182, 109577868, 109579799, 109586060, 109618842, 109642728]),
 array([109576013, 109578017, 109580002, 109586162, 109619039, 109643675])]

In [75]:
#free unused space
del all_introns

In [173]:
gene_data = [v for k, v in best_transcripts.values()]

In [228]:
# Now we gotta calculate introns


def get_introns(gene):
    # First, we figure out the start and end indexes 
    x = gene #best_transcripts['ENSG00000147642'][1]#exons[0]
    tr_start = x[3]
    tr_end = x[4]
    direction = x[5]
    ex_starts = x[7]
    ex_ends = x[8]
    transcript = x[9]

    ex_starts = ex_starts - tr_start
    ex_ends = ex_ends - tr_start
    tr_end = tr_end - tr_start
    tr_start = 0
    
#     print(ex_starts)
#     print(ex_ends)

    if direction == -1:
#         print('flipped')
        ex_starts = np.flip(tr_end - ex_starts)
        ex_ends = np.flip(tr_end - ex_ends)
        tmp = ex_ends
        ex_ends = ex_starts
        ex_starts = tmp

    in_starts = ex_ends[:-1]+1
    in_ends = ex_starts[1:]
    
#     print(in_starts)
#     print(in_ends)

    introns = []

    for i in range(len(in_starts)):
        intron = transcript[in_starts[i]:in_ends[i]]
#         print(intron)
        introns.append(intron)
        
    return introns
    
    

In [217]:
with open('introns.csv', 'w') as f:
    for i in tqdm(range(len(gene_data)), position=0, leave=True):
        gene = gene_data[i]
        introns = get_introns(gene)
        for i, intron in enumerate(introns):
            # gene, transcript, transcript bp, intron #, 
            f.write(gene[0] + ',' + gene[2] + ',' + str(gene[6]) + ','
                    + str(i+1) + ',' + intron + '\n')

100%|██████████| 67140/67140 [00:20<00:00, 3206.61it/s] 


In [230]:
# Sanity check
# q = open('introns.csv', 'r')
# for i in range(6):
#     print(q.readline())